# ESERCIZIO 4
Exercise 4. How can we parameterize our model to replicate the results shown in figure 11 of Nowak and May (1993)?

**1) Obiettivo dell'esercizio**

Questo modello simula la **diffusione della defezione in una popolazione inizialmente cooperante**, dove **un solo agente al centro è defezionatore**. È un gioco del dilemma del prigioniero su griglia spaziale, ispirato alla Figura 11 del paper di Nowak e May (1993)

**Lo scopo è osservare come un comportamento non cooperativo può diffondersi nel tempo, creando strutture complesse e dinamiche.**


#**Il dilemma del prigioniero**

Nel modello utilizzato per l’Esercizio 4 (Figura 11 di Nowak & May, 1993), ogni agente gioca il Dilemma del Prigioniero con i propri vicini. Questo gioco classico della teoria dei giochi mette in evidenza il conflitto tra interesse individuale e collaborazione.

**2) Il codice**

Gli agenti sono distribuiti su una griglia 201x201 e giocano ripetutamente il dilemma del prigioniero con i vicini (vicinato di tipo Von Neumann, cioè 4 direzioni + sé stessi).
**All’inizio, tutti gli agenti sono cooperatori, tranne uno al centro della griglia, che parte come defezionatore. A ogni turno, tutti calcolano il proprio guadagno interagendo con i vicini, poi copiano la strategia del vicino col miglior payoff. Questo aggiornamento è sincrono, cioè avviene contemporaneamente per tutti.**

Il codice genera 95 fotogrammi, ciascuno rappresentante lo stato della griglia a ogni passo. Le celle nere indicano cooperatori, quelle bianche defezionatori. L’animazione risultante mostra come, da un singolo defezionatore iniziale, si sviluppi una struttura simmetrica che si espande e invade la popolazione, formando motivi frattali molto riconoscibili. Il risultato evidenzia la vulnerabilità della cooperazione quando la defezione è sufficientemente vantaggiosa: **anche partendo da una popolazione totalmente cooperativa, un singolo defezionatore può generare un cambiamento irreversibile.**


**Infine, tutti aggiornano insieme la loro strategia. Questo processo si ripete per molti cicli: nel tempo, la strategia del defezionatore si diffonde e crea strutture geometriche complesse, che vengono visualizzate in un’animazione**



**3) Risultati finali**

Il defezionatore iniziale viene imitato, formando macchie bianche che si espandono.


Il risultato finale:
- **Bianco:** defezionatori

- **Nero:** cooperatori



In [ ]:
# 1) Installa Mesa e librerie di animazione
!pip install mesa==3.1.1 -q

# 2) Import
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from mesa import Agent, Model
from mesa.space import MultiGrid
import numpy as np, random
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# 3) Definizione Agent
#Definisce cosa fa ogni singolo agente (una cella della griglia):

class PatchAgent(Agent):
    def __init__(self, model, pos):
        super().__init__(model)
        self.pos = pos
        self.strategy = 0
        self.next_strategy = 0
        self.payoff = 0

    def compute_neighbors(self):
        self.neigh = self.model.grid.get_neighbors(
            self.pos,
            moore=False,          # Von Neumann
            include_center=True,  # include self
            radius=1
        )

    def play(self):
        self.compute_neighbors()
        # conto quanti cooperanti (0) e defezionatori (1)
        c0 = sum(1 for a in self.neigh if a.strategy == 0)
        c1 = len(self.neigh) - c0
        # payoff_matrix = [[1,0],[b,0]]  #dilemma del prigioniero
        p = self.model.payoff_matrix[self.strategy]
        self.payoff = p[0]*c0 + p[1]*c1

    #copia la strategia del vicino
    def revise(self):
        # best‐neighbor (incluso sé stesso)
        best = max(self.neigh, key=lambda a: a.payoff)
        self.next_strategy = best.strategy
  #la aggiorna
    def update(self):
        self.strategy = self.next_strategy

# 4) Definizione Model
class GameModel(Model):
    def __init__(self, size, b, seed=0):
        super().__init__(seed=seed)
        random.seed(seed); np.random.seed(seed)
        # PD spaziale: R=1, S=0, T=b, P=0
        self.payoff_matrix = np.array([[1,0],[b,0]])
        self.grid = MultiGrid(size, size, torus=True)
        # piazzo gli agenti
        for x in range(size):
            for y in range(size):
                a = PatchAgent(self, (x, y))
                self.grid.place_agent(a, (x, y))

        # inizializzo tutti cooperanti (0) tranne uno al centro
        center = (size//2, size//2)
        for a in self.agents:
            a.strategy = 0
        # trovo l'agente in centro e lo imposto a defezionatore
        for a in self.grid.get_cell_list_contents([center]):
            a.strategy = 1


    def step(self):
        # fase di gioco
        self.agents.do("play")
        # fase di revisione
        self.agents.do("revise")
        # aggiornamento sincrono
        self.agents.do("update")

# 5) Simulazione + cattura 96 frame
#ad ogni turno, gli agenti giocano, confrontano i guadagni, copiano la strategia migliore tra i vicini e aggiornano la loro
def make_animation(size=201, b=1.8, steps=95, interval=200):
    model = GameModel(size, b=b, seed=42)
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set_xticks([]); ax.set_yticks([])

    # inizializzo il plot col frame t=0
    grid = np.zeros((size, size))
    for a in model.agents:
        x,y = a.pos
        grid[x,y] = a.strategy
    im = ax.imshow(grid.T, origin='lower', cmap='Greys', vmin=0, vmax=1)

    def update_frame(t):
        model.step()
        # aggiorno la matrice delle strategie
        for a in model.agents:
            x,y = a.pos
            grid[x,y] = a.strategy
        im.set_data(grid.T)
        ax.set_title(f"t = {t+1}")
        return [im]

    ani = animation.FuncAnimation(
        fig, update_frame, frames=steps, blit=True, interval=interval
    )
    plt.close(fig)
    return HTML(ani.to_html5_video())

# 6) Esegui l’animazione (t=0…95) con b in (5/3,2)
make_animation(size=201, b=1.8, steps=95, interval=200)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 kB 3.1 MB/s eta 0:00:00


#**Risultato finale**

Il risultato finale mostra come un singolo comportamento deviante (defezione) può, grazie alla struttura spaziale e alla dinamica di imitazione, invadere l’intera popolazione cooperativa, dando origine a pattern regolari.

**Col passare dei turni, la strategia del defezionatore si diffonde, perché defezionare conviene di più se gli altri cooperano.**

Questo dimostra che **anche in contesti spaziali, la cooperazione può essere instabile se i vantaggi della defezione sono troppo alti.**